In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt

train = pd.read_csv('./data.csv')
test = pd.read_csv('./test.csv')
test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.56,Ideal,I,VS2,62.1,54.0,5.30,5.33,3.30
1,1,1.05,Ideal,G,VS2,61.9,56.0,6.56,6.52,4.05
2,2,0.50,Premium,E,VS2,61.5,56.0,5.11,5.07,3.13
3,3,0.80,Ideal,F,VS1,62.8,56.0,5.91,5.96,3.73
4,4,0.54,Ideal,G,VS1,61.4,57.0,5.23,5.20,3.20


In [2]:
Y = train['price']
X = train[['carat', 'cut', 'color', 'clarity','depth', 'table','x', 'y', 'z']]

In [3]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=35)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

In [4]:
#limpio el modelo de entrenamiento

def cleaningtrain(training):
    X = training.copy()
    X['volume'] = X['x'] * X['y'] * X['z']
    X.drop(['z','y','x'], axis=1, inplace=True)
    X = pd.get_dummies(X, columns = ['color', 'cut', 'clarity'], drop_first=True)
    #X['carat'] = X['carat']*10
    
    return X
    
    

In [5]:
#limpio el modelo de test

def cleaningtest(testing): 
    Xtest = testing.copy()
    #Xtest['volume'] = Xtest['x'] * Xtest['y'] * Xtest['z']
    Xtest.drop(['z','y','x'], axis=1, inplace=True)
    Xtest = pd.get_dummies(X, columns = ['color', 'cut', 'clarity'], drop_first=True)
    #Xtest['carat'] = Xtest['carat']*2
    
      
    return Xtest
    

In [6]:
xx = cleaningtrain(X)
xx.head()

,carat,depth,table,volume,color_E,color_F,color_G,color_H,color_I,color_J,...,cut_Ideal,cut_Premium,cut_Very Good,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,2.26,61.9,57.0,366.903680,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
1,2.43,63.2,57.0,392.176400,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
2,0.80,61.0,57.0,133.001901,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,0.40,63.3,60.0,64.059840,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0.31,61.6,55.0,51.797610,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [7]:
#normalizando el train
'''
def normalize(df, columns):
    mean=[]
    std=[]
    for column in columns:
        mean.append(df[column].mean())
        std.append(df[column].std())
        df[column]=(df[column]-mean[-1])/std[-1]
    return df,mean,std
def normalize_params(df, columns,mean,std):
    for i in range(len(columns)):
        df[columns[i]]=(df[columns[i]]-mean[i])/std[i]
    return df
'''

'\ndef normalize(df, columns):\n    mean=[]\n    std=[]\n    for column in columns:\n        mean.append(df[column].mean())\n        std.append(df[column].std())\n        df[column]=(df[column]-mean[-1])/std[-1]\n    return df,mean,std\ndef normalize_params(df, columns,mean,std):\n    for i in range(len(columns)):\n        df[columns[i]]=(df[columns[i]]-mean[i])/std[i]\n    return df\n'

In [8]:
print(Y.shape)
print(xx.shape)

(40455,)
(40455, 21)


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xx, Y, test_size=0.2, random_state=35)

In [10]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

models = {
'RandomForestRegressor1': RandomForestRegressor(min_samples_leaf=1,min_samples_split=3,n_estimators=200),
#'RandomForestRegressor2': RandomForestRegressor(min_samples_leaf=2,min_samples_split=10,n_estimators=200),
#'RandomForestRegressor3': RandomForestRegressor(min_samples_leaf=4,min_samples_split=7,n_estimators=200),
#'RandomForestRegressor4': RandomForestRegressor(min_samples_leaf=6,min_samples_split=10,n_estimators=100),
#'RandomForestRegressor5': RandomForestRegressor(min_samples_leaf=10,min_samples_split=5,n_estimators=150),
}


In [11]:
for label,model in models.items():
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    y_score = model.score(X_train,y_train)
    mae = round(mean_absolute_error(y_test, y_pred),2)
    mse = mean_squared_error(y_test, y_pred)
    auc = r2_score(y_test, y_pred)
    print('Model={} Score={} mae={} mse={} auc={}'.format(label, y_score, mae, mse**0.5, auc))

Model=RandomForestRegressor1 Score=0.9957877138983188 mae=306.87 mse=635.5274787185366 auc=0.9752436330677415
